## 네이버 검색 API 활용

In [2]:
# API 접근을 위한 클라이언트 정보 설정
client_id = "YOUR_API_ID"
client_secret = "YOUR_API_KEY"

### 블로그 검색
(1) 네이버 개발자 센터에서 제공하는 기본 코드

In [ ]:
import os
import sys
import urllib.request
encText = urllib.parse.quote("인공지능") #UTF-8
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

(2) requests 라이브러리를 통한 호출

In [ ]:
# requests 라이브러리 임포트
import requests
from urllib.parse import quote

# 검색어 설정
search_query = "인공지능"

# API 기본 URL
base_url = "https://openapi.naver.com/v1/search/blog"

# API 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# API 요청 파라미터 설정
params = {
    "query": search_query,      # 검색어
    "display": 10,             # 검색 결과 출력 건수 (기본값: 10, 최대: 100)
    "start": 1,                # 검색 시작 위치 (기본값: 1, 최대: 1000)
    "sort": "sim"              # 정렬 방식 - sim: 정확도순, date: 날짜순
}


# GET 요청 보내기 (params 딕셔너리 사용)
response = requests.get( # 한글 자동으로 UTF-8인코딩 함
    base_url,
    headers=headers,
    params=params
)

# 응답 상태 코드 확인 및 결과 출력
if response.status_code == 200:
    # JSON 형식으로 응답 데이터 파싱
    result = response.json()
    print(result)
        
    # 전체 검색 결과 수 출력
    print(f"총 검색 결과: {result['total']} 건")
    print(f"현재 페이지: {params['start']} / {result['total']}")
    print("-" * 50)
    
    # 검색 결과 출력
    for item in result.get('items', []):
        print(f"제목: {item['title']}")
        print(f"블로거: {item['bloggername']}")
        print(f"작성일: {item['postdate']}")
        print(f"링크: {item['link']}")
        print(f"설명: {item['description']}")
        print("-" * 50)
else:
    # 에러 발생시 상태 코드와 에러 메시지 출력
    print(f"Error Code: {response.status_code}")
    print(f"Error Message: {response.text}")

### 이미지 검색
(1) 기본 요청 받기

In [ ]:
# requests 라이브러리 임포트
import requests
from urllib.parse import quote


# 검색어 설정 및 UTF-8 인코딩
search_query = "고양이"

# API 기본 URL (이미지 검색 URL)
base_url = "https://openapi.naver.com/v1/search/image"

# API 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# API 요청 파라미터 설정
params = {
    "query": search_query,     # 검색어 (필수)
    "display": 20,             # 검색 결과 출력 건수 (기본값: 10, 최대: 100)
    "start": 1,                # 검색 시작 위치 (기본값: 1, 최대: 1000)
    "sort": "sim",             # 정렬 방식 - sim: 정확도순, date: 날짜순
    "filter": "small"            # 이미지 크기 필터 - all, large, medium, small
}

# GET 요청 보내기
response = requests.get( # 한글 자동으로 UTF-8인코딩 함
    base_url,
    headers=headers,
    params=params
)

# 응답 상태 코드 확인 및 결과 출력
if response.status_code == 200:
    # JSON 형식으로 응답 데이터 파싱
    result = response.json()
    
    # 검색어 및 전체 검색 결과 수 출력
    print(f"검색어: {search_query}")
    print(f"총 검색 결과: {result['total']} 건")
    print(f"현재 페이지: {params['start']} / {result['total']}")
    print("-" * 50)
    
    # 검색 결과 출력
    for index, item in enumerate(result.get('items', []), 1):
        print(f"[이미지 {index}]")
        print(f"제목: {item['title'].replace('<b>', '').replace('</b>', '')}")
        print(f"썸네일 URL: {item['thumbnail']}")
        print(f"원본 이미지 URL: {item['link']}")
        print(f"이미지 크기: {item['sizewidth']}x{item['sizeheight']}")
        print("-" * 50)

else:
    # 에러 발생시 상태 코드와 에러 메시지 출력
    print(f"Error Code: {response.status_code}")
    print(f"Error Message: {response.text}")

(2) 이미지 다운로드

In [ ]:
# 필요한 라이브러리 임포트
import requests
from urllib.parse import quote
import os
import time
from datetime import datetime
import re

def get_filename(filename):
    """파일명에 사용할 수 없는 특수문자를 제거하는 함수"""
    # Windows에서 파일명으로 사용할 수 없는 문자들을 제거 또는 대체
    return re.sub(r'[\\/:*?"<>|]', '', filename)

def download_image(url, folder_path, filename):
    """이미지를 다운로드하는 함수"""
    try:
        response = requests.get(url, stream=True, timeout=10)
        if response.status_code == 200:
            # 파일 확장자 추출 (없으면 .jpg로 기본 설정)
            content_type = response.headers.get('content-type', '')
            if 'jpeg' in content_type or 'jpg' in content_type:
                ext = '.jpg'
            elif 'png' in content_type:
                ext = '.png'
            elif 'gif' in content_type:
                ext = '.gif'
            else:
                ext = '.jpg'

            # 최종 파일 경로 생성
            final_path = os.path.join(folder_path, f"{filename}{ext}")
            
            # 이미지 저장
            with open(final_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            return True, final_path
        return False, "HTTP Error: " + str(response.status_code)
    except Exception as e:
        return False, str(e)

# 검색어 설정 및 UTF-8 인코딩
search_query = "고양이"

# 검색 결과 저장을 위한 폴더 생성
base_folder = "downloaded_images"
folder_path = os.path.join(base_folder, search_query)

# 폴더가 없으면 생성
os.makedirs(folder_path, exist_ok=True)

# API 기본 URL (이미지 검색 URL)
base_url = "https://openapi.naver.com/v1/search/image"

# API 요청 헤더 설정
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# API 요청 파라미터 설정
params = {
    "query": search_query,     # 검색어 (필수)
    "display": 30,             # 검색 결과 출력 건수 (기본값: 10, 최대: 100)
    "start": 1,                # 검색 시작 위치 (기본값: 1, 최대: 1000)
    "sort": "sim",             # 정렬 방식 - sim: 정확도순, date: 날짜순
    "filter": "medium"          # 이미지 크기 필터 - all, large, medium, small
}

# GET 요청 보내기
response = requests.get(
    base_url,
    headers=headers,
    params=params
)

# 다운로드 결과 통계
download_stats = {
    'success': 0,
    'failed': 0,
    'total': 0
}

# 응답 상태 코드 확인 및 결과 출력
if response.status_code == 200:
    # JSON 형식으로 응답 데이터 파싱
    result = response.json()
    
    print(f"검색어: {search_query}")
    print(f"총 검색 결과: {result['total']} 건")
    print(f"다운로드 폴더: {folder_path}")
    print("-" * 50)
    
    # 검색 결과 처리 및 이미지 다운로드
    for index, item in enumerate(result.get('items', []), 1):
        print(f"[이미지 {index}/{len(result['items'])}] 다운로드 중...")
        print(f"제목: {item['title'].replace('<b>', '').replace('</b>', '')}")
        
        # 파일명 생성 (제목 + 인덱스)
        title_part = get_filename(item['title'][:50].replace('<b>', '').replace('</b>', ''))
        filename = f"{index:03d}_{title_part}"
        
        # 이미지 다운로드 시도
        success, message = download_image(item['link'], folder_path, filename)
        
        if success:
            print(f"다운로드 완료: {message}")
            download_stats['success'] += 1
        else:
            print(f"다운로드 실패: {message}")
            download_stats['failed'] += 1
        
        download_stats['total'] += 1
        print("-" * 50)
        
        # 네이버 서버 부하 방지를 위한 딜레이
        time.sleep(0.5)

    # 다운로드 통계 출력
    print("\n[다운로드 완료 통계]")
    print(f"성공: {download_stats['success']} 건")
    print(f"실패: {download_stats['failed']} 건")
    print(f"총 시도: {download_stats['total']} 건")
    print(f"성공률: {(download_stats['success']/download_stats['total']*100):.1f}%")
    print(f"\n이미지 저장 위치: {folder_path}")

else:
    # 에러 코드 및 메시지 출력
    error_codes = {
        400: "SE01: 잘못된 쿼리요청입니다.",
        403: "API 권한이 없습니다. 개발자 센터에서 API 권한을 확인하세요.",
        404: "SE05: 존재하지 않는 검색 API입니다.",
        500: "SE99: 시스템 에러"
    }
    
    error_message = error_codes.get(response.status_code, "알 수 없는 에러가 발생했습니다.")
    print(f"Error Code: {response.status_code}")
    print(f"Error Message: {error_message}")
    print(f"서버 응답: {response.text}")